<a href="https://colab.research.google.com/github/vaghyjuli/RL/blob/main/go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/RLP
! pip install gym
! git clone https://github.com/aigagror/GymGo
! git pull
%cd GymGo
! pip install -e .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/RLP
fatal: destination path 'GymGo' already exists and is not an empty directory.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/content/drive/My Drive/RLP/GymGo
Obtaining file:///content/drive/My%20Drive/RLP/GymGo
  Running setup.py develop for gym-go


N = board size

# **Action space**
env.action_space = Discrete(N * N + 1) \\
board moves = tuple (0 to N-1, 0 to N-1) \\
pass = N * N - 1 \\
(game ends when both players pass)

# **State space**
env.observation_space = Box(0.0, 6.0, (6, N, N), float32)

6 x N x N numpy array -> all values in the array are either 0 or 1

[0] black pieces \\
[1] white pieces \\
[2] indicator layer for whose turn it is \\
[3] invalid moves (including ko-protection) for the next action \\
[4] indicator layer for whether the previous move was a pass \\
[5] indicator layer for whether the game is over

In [ ]:
import gym
import numpy as np

In [ ]:
class Player():
  def __init__(self, N):
    self.N = N
    self.policy = np.ones((N, N)) / (N*N)

  def get_action(self, state):
    allowed_actions = np.absolute(state[3] - 1) # 1=valid, 0=invalid
    if sum(sum(allowed_actions)) == 0:          # pass if there are no allowed actions
      return self.N * self.N
    allowed_action_prob = np.multiply(self.policy, allowed_actions)
    allowed_action_prob /= sum(sum(allowed_action_prob))
    action = self.random_choice_2D(allowed_action_prob)
    return action

  def random_choice_2D(self, prob_2D):
    """
    np.random.choice only works for 1D arrays
    return tuple (row, col)
    """
    row_prob = np.sum(prob_2D, axis = 1)
    row = np.random.choice(range(self.N), p = row_prob)
    col_prob = prob_2D[row] / sum(prob_2D[row])
    col = np.random.choice(range(self.N), p = col_prob)
    return (row, col)

In [ ]:
N = 7
env = gym.make('gym_go:go-v0', size=N, komi=0, reward_method='heuristic')
player = Player(N)

In [ ]:
state = env.reset()
for i in range(1):
  action = player.get_action(state)
  print(f"Step {i}\n{action}")
  state, reward, done, info = env.step(action)
  print(env.render("terminal"))
  if done:
    print("Game over")
    break
env.close()

Step 0
(6, 1)
    0   1   2   3   4   5   6
  -----------------------------
0 | . | . | . | . | . | . | . |
  -----------------------------
1 | . | . | . | . | . | . | . |
  -----------------------------
2 | . | . | . | . | . | . | . |
  -----------------------------
3 | . | . | . | . | . | . | . |
  -----------------------------
4 | . | . | . | . | . | . | . |
  -----------------------------
5 | . | . | . | . | . | . | . |
  -----------------------------
6 | . | B | . | . | . | . | . |
  -----------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 49.0, White Area: 0.0

None
